In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [17]:
# Open the files and convert to dataframes
df_train = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv', sep = ",", header = 0, index_col = 0).
df_test = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv', sep = ",", header = 0, index_col = 0)

# Explore dataset
df_train.head()

In [4]:
df_train.describe()


Using information from: https://towardsdatascience.com/7-ways-to-handle-missing-values-in-machine-learning-1a6326adf79e
<br>Handling of Null Values:
<br>1) if columns has 50% misisng values, drop them
<br>2) Impute missing values with mean/median or 0, depending on the data in column.
<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; - or can use interpolation for time series dataset, Last observation carried forward (LOCF) method for longitudinal behaviours
<br>3) For categorical data, if missing values are minimal, replace wiht most frequent, if not replace with a new cateogry variable
<br>4) Use of ML algorithms: KNN or navies Bayes (BUT scikit does not support), or Random Forest
<br>5) Regression or Classification models (uses correlation advantage as compared to (4))
<br>6) Deep Learning Library (Datawig)using deep neural networks to predict missing values
<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; - need to learn more on this

In [5]:
over50_null = (df_train.isna().sum()/1460)[df_train.isna().sum()/1460 > 0.5].index.values.tolist()
df_train.drop(columns=(over50_null), inplace = True)
df_test.drop(columns=(over50_null), inplace = True)

df_train

In [6]:
# Check for the rest of null values
df_train.isna().any()[lambda x: x].index.values.tolist()


In [7]:
df_test.isna().any()[lambda x: x].index.values.tolist()

In [8]:
# Separate columns to numerical and categorical for imputation of missing values
numerical = df_train._get_numeric_data().columns
categorical = list(set(df_train.columns) - set(numerical))

In [9]:
# numerical without the SalePrice (answer)
numerical = numerical.drop("SalePrice")
numerical

In [10]:
# categorical variables
categorical

In [11]:
# Fill up all na categorical missing values with None as it indicates that there aren't any measurements to get
def fill_data_cat(data):
    
    for col in categorical:
        data.loc[data[col].isna(), col] = 'None'
    
    return data

In [12]:
# Categorical column missing values filled
df_train_CatFilled = fill_data_cat(df_train)
df_test_CatFilled = fill_data_cat(df_test)

# Checking if all categorical columns filled
print(df_train_CatFilled.isna().any()[lambda x: x])
print(df_test_CatFilled.isna().any()[lambda x: x])

In [13]:
# numerical values to be 0 as nothing to measure (time data might have issues)
def fill_data_num(data):
    for col in numerical:
        data.loc[data[col].isna(), col] = '0'


In [14]:
# Numerical column missing values filled
df_train_filled = fill_data_num(df_train_CatFilled)
df_test_filled = fill_data_num(df_test_CatFilled)

# Checking if all columns filled now
print(df_train_CatFilled.isna().any()[lambda x: x])
print(df_test_CatFilled.isna().any()[lambda x: x])

In [15]:
# Now we have settled all missing values
# There are still lots of dimensions (columns) so we will try to reduce 
df_train_CatFilled
